In [ ]:
import os

import gymnasium as gym
import panda_gym

from huggingface_sb3 import load_from_hub, package_to_hub

from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env

from huggingface_hub import notebook_login

In [ ]:
# from pyvirtualdisplay import Display

# virtual_display = Display(size=(800, 600))
# virtual_display.start()

In [ ]:
# create environment
import pprint

env_id = "PandaReachDense-v3"
env = gym.make(env_id)

# state & action
s_size = env.observation_space.shape
a_size = env.action_space.shape

print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", s_size)
# print("Sample observation", env.observation_space.sample())  # Get a random observation
obs1 = env.observation_space.sample()
pprint.pp(obs1)

In [ ]:
print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", a_size)
print("Action Space Sample", env.action_space.sample())  # Take a random action

In [ ]:
env = make_vec_env(env_id, n_envs=4)
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.0)

model = A2C(policy="MultiInputPolicy", env=env, verbose=1)
model.learn(50_000, progress_bar=True)

model.save("a2c_panda_reach")
env.save("vec_normalize.pkl")

In [ ]:
# Evaluate the agent

from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

eval_env = DummyVecEnv([lambda: gym.make(env_id)])
eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)

eval_env.render_mode = "rgb_array"
eval_env.training = False
eval_env.norm_reward = False  # not need at test time

model = A2C.load("a2c_panda_reach")
mean_reward, std_reward = evaluate_policy(model, eval_env)

print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")